## 1. Kensu initialisation

To use the library, you need to add the kensu jar to the spark client

Initialize the client with the `KensuProvider` object with its `Context`:
- `process_name`:  the application name
- `project_names`: where the application is running 
- `environment`: ... well

In [ ]:
%%init_spark
launcher.jars = ["libs/kensu-dam-spark-collector-0.17.2_spark-3.0.1.jar","libs/sdk_2.12.jar"]
launcher.conf.set("spark.sql.shuffle.partitions", "1")

In [ ]:
val app = 2
val week = 1

In [ ]:
implicit val ch = new io.kensu.dodd.sdk.ConnectHelper(s"./conf/application${app}-week${week}.properties")
val dataFolder = "data"+ch.properties("dam.activity.user").toString
io.kensu.third.integration.TimeUtils.setMockedTime(ch.properties("mocked.timestamp").asInstanceOf[Long], false)
io.kensu.dam.lineage.spark.lineage.Implicits.SparkSessionDAMWrapper(spark).track(ch.properties.get("dam.ingestion.url").map(_.toString), None)(ch.properties.toList:_*)

 ## 2. Execution of the pipeline - App 2 : Total
 
 This application reads the parquet file created by App1 and computes a KPI : `Total`



In [ ]:
val df = spark.read.option("inferSchema","true").option("header","true").load(s"./data_${properties("dam.activity.user")}/week${week}/joined_data")

In [ ]:
df.show()

In [ ]:
df.createOrReplaceTempView("df")

In [ ]:
val Total_DF = spark.sql("SELECT id,price,quantity,round(price * quantity,2) AS total FROM df")

In [ ]:
Total_DF.show()

In [ ]:
val sdk = new io.kensu.dodd.sdk.SDK(ch.properties("kensu.sdk.url").toString, ch.properties("kensu.pat").toString)

val rule = sdk.range(Some(4),None)

sdk.addRule("data_total","id.nullrows",rule)


In [ ]:
Total_DF.write.mode("overwrite").save(s"./data_${properties("dam.activity.user")}/week${week}/data_total")